# Recurent Neural Network V1 Test 1

### Travail effectué
* RNN avec 3 couches

In [17]:
import pandas as pd
from matplotlib import pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from sklearn.metrics import classification_report
from Fonctions_utils import f1_m, matrix_confusion

In [18]:
df_train = pd.read_csv("../../data_models/df_train.csv")
df_val = pd.read_csv("../../data_models/df_val.csv")
df_test = pd.read_csv("../../data_models/df_test.csv")

In [19]:
sentences_train = df_train["text_clean"].values.astype(str)
sentences_val = df_val["text_clean"].values.astype(str)
sentences_test = df_test["text_clean"].values.astype(str)

y_train = df_train["Insult"].values
y_val = df_val["Insult"].values
y_test = df_test["Insult"].values

In [20]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_val = tokenizer.texts_to_sequences(sentences_val)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1

The indexing is ordered after the most common words in the text, which you can see by the word the having the index 1. It is important to note that the index 0 is reserved and is not assigned to any word. This zero index is used for padding, which I’ll introduce in a moment

One problem that we have is that each text sequence has in most cases different length of words. To counter this, you can use pad_sequence() which simply pads the sequence of words with zeros. By default, it prepends zeros but we want to append them. Typically it does not matter whether you prepend or append zeros.

In [21]:
maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [22]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.LSTM(units=32, activation='tanh'))
model.add(layers.Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=[f1_m, 'accuracy'])

In [24]:
history = model.fit(X_train, y_train,
                    epochs=30,
                    validation_data=(X_val, y_val),
                    verbose=2,
                    batch_size=10)

Train on 2762 samples, validate on 1185 samples
Epoch 1/50


2023-05-28 20:16:20.528884: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference___backward_standard_lstm_69536_70021_specialized_for_StatefulPartitionedCall_at___inference_distributed_function_70205' and '__inference___backward_cudnn_lstm_with_fallback_69203_69385' both implement 'lstm_82fae96b-c331-4f6d-808e-434ace2a9fa2' but their signatures do not match.


2760/2762 [============================>.] - ETA: 0s - loss: 0.5846 - accuracy: 0.7348

2023-05-28 20:17:11.658053: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference_standard_lstm_71236_specialized_for_sequential_3_lstm_2_StatefulPartitionedCall_at___inference_distributed_function_71595' and '__inference_cudnn_lstm_with_fallback_71347' both implement 'lstm_59359fcd-741d-4dde-b95d-c9d529ec3f7e' but their signatures do not match.


2762/2762 [==============================] - 58s 21ms/sample - loss: 0.5844 - accuracy: 0.7350 - val_loss: 0.5930 - val_accuracy: 0.7325
Epoch 2/50
2762/2762 [==============================] - 52s 19ms/sample - loss: 0.5809 - accuracy: 0.7350 - val_loss: 0.5847 - val_accuracy: 0.7325
Epoch 3/50
2762/2762 [==============================] - 53s 19ms/sample - loss: 0.5802 - accuracy: 0.7350 - val_loss: 0.5914 - val_accuracy: 0.7325
Epoch 4/50
2762/2762 [==============================] - 53s 19ms/sample - loss: 0.5796 - accuracy: 0.7350 - val_loss: 0.5809 - val_accuracy: 0.7325
Epoch 5/50
2762/2762 [==============================] - 54s 20ms/sample - loss: 0.5810 - accuracy: 0.7350 - val_loss: 0.5814 - val_accuracy: 0.7325
Epoch 6/50
2762/2762 [==============================] - 52s 19ms/sample - loss: 0.5800 - accuracy: 0.7350 - val_loss: 0.5810 - val_accuracy: 0.7325
Epoch 7/50
2762/2762 [==============================] - 52s 19ms/sample - loss: 0.5793 - accuracy: 0.7350 - val_loss: 0.580

: 

: 

In [ ]:
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
plot_history(history)

In [ ]:
y_predict_test = model.predict(X_test)
y_predict_test2 = [round(item) for sublist in y_predict_test.tolist() for item in sublist]

In [ ]:
matrix_confusion(y_test, y_predict_test2)

In [ ]:
target_names = ['Neutre', 'Insultant']
print(classification_report(y_test, y_predict_test2, target_names=target_names))